In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch(['http://localhost:9200'])

In [5]:
import eland as ed

In [10]:
import pandas as pd

In [12]:
df = pd.DataFrame(
    {
        "process.name": ["process1"],
        "file.hash.sha256": ["hash1"],
        "label": [1],
    }
)

In [13]:
index = "test_mapping"

In [15]:
ed.pandas_to_eland(
    df,
    es_client,
    es_dest_index=index,
    es_if_exists="append",
    es_refresh=True,
    use_pandas_index_for_es_ids=False
)

/Users/sidhuas/.pyenv/versions/3.9.1/envs/eland/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
/Users/sidhuas/.pyenv/versions/3.9.1/envs/eland/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


,file.hash.sha256,label,process.name
FabFA34BeVG4RuLOof0E,hash1,1,process1


In [16]:
ed.pandas_to_eland(
    df,
    es_client,
    es_dest_index=index,
    es_if_exists="append",
    es_refresh=True,
    use_pandas_index_for_es_ids=False
)

ValueError: DataFrame dtypes and Elasticsearch index mapping aren't compatible:
- 'file' is missing from DataFrame columns
- 'process' is missing from DataFrame columns
- 'file.hash.sha256' is missing from ES index mapping
- 'process.name' is missing from ES index mapping